In [3]:
from dbutil import conn
import pyTigerGraph as tg
import json
from pprint import pprint


In [4]:
print(conn.gsql('ls', options=[]))

---- Graph propgraph
Vertex Types:
- VERTEX property(PRIMARY_ID mls_id STRING, address STRING, city STRING, state STRING, zip STRING, price FLOAT, lat DOUBLE, lng DOUBLE, beds FLOAT, baths FLOAT, sqft FLOAT, url STRING, type STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE"
- VERTEX trait(PRIMARY_ID name STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE"
Edge Types:
- UNDIRECTED EDGE comparable(FROM property, TO property, reason STRING, created_at DATETIME)
- UNDIRECTED EDGE has_trait(FROM property, TO trait)

Graphs:
- Graph propgraph(property:v, trait:v, comparable:e, has_trait:e)
Jobs:
Queries:
- comp_search(vertex<property> p) (installed v2)






In [16]:
if True:
    print(conn.gsql('''
        use global
        drop query comp_search
        drop edge has_trait
        drop edge comparable
        drop vertex property
        drop vertex trait
        drop graph propgraph
        '''))

Semantic Check Fails: These queries could not be found anywhere: [comp_search].
Semantic Check Fails: The edge type 'has_trait' could not be found.
Failed to drop edge types: [has_trait].
Semantic Check Fails: The edge type 'comparable' could not be found.
Failed to drop edge types: [comparable].
Semantic Check Fails: The vertex type 'property' could not be found.
Failed to drop vertex types: [property].
Successfully dropped vertex types: [trait].
The graph propgraph could not be dropped!


In [17]:
# Type, address, city, state, zip, price, beds, baths, location, square feet, lot size, year built, days on market, $/square feet, hoa/month, latitude, longitude
print(conn.gsql('''
create vertex property (primary_id mls_id string, address string, city string, state string, zip string, price float, lat double, lng double, beds float, baths float, sqft float, url string, type string)
                      
create undirected edge comparable(from property, to property, reason string, created_at datetime)

create vertex trait(primary_id name string)

create undirected edge has_trait(from property, to trait)

create graph propgraph(property, comparable, trait, has_trait)
''', options=[]))

Successfully created vertex types: [property].
Successfully created edge types: [comparable].
Successfully created vertex types: [trait].
Successfully created edge types: [has_trait].
The graph propgraph is created.


In [15]:
print(conn.gsql('ls', options=[]))

---- Global vertices, edges, and all graphs
Vertex Types:
- VERTEX trait(PRIMARY_ID name STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE"
Edge Types:

Graphs:
Jobs:


JSON API version: v2
Syntax version: v2



In [18]:
sample_properties = conn.gsql('select * from property limit 10')
sample_properties

'[\n{\n"v_id": "72917596",\n"attributes": {\n"zip": "02127",\n"baths": 2,\n"address": "420 W Broadway #306",\n"lng": -71.04722,\n"city": "Boston",\n"price": 954500,\n"sqft": 968,\n"state": "MA",\n"beds": 2,\n"type": "Condo/Co-op",\n"lat": 42.33674,\n"url": "https://www.redfin.com/MA/Boston/420-W-Broadway-02127/unit-306/home/175036534"\n},\n"v_type": "property"\n},\n{\n"v_id": "72877110",\n"attributes": {\n"zip": "02127",\n"baths": 1,\n"address": "202-206 K St #3",\n"lng": -71.03725,\n"city": "Boston",\n"price": 440000,\n"sqft": 530,\n"state": "MA",\n"beds": 1,\n"type": "Condo/Co-op",\n"lat": 42.33638,\n"url": "https://www.redfin.com/MA/Boston/202-K-St-02127/unit-3/home/21719102"\n},\n"v_type": "property"\n},\n{\n"v_id": "72785349",\n"attributes": {\n"zip": "02126",\n"baths": 3,\n"address": "14 Lena Ter #2",\n"lng": -71.08534,\n"city": "Boston",\n"price": 660000,\n"sqft": 3100,\n"state": "MA",\n"beds": 4,\n"type": "Townhouse",\n"lat": 42.2779,\n"url": "https://www.redfin.com/MA/Boston/1

In [6]:
"""
Usually comparables just go one layer deep i.e. A -> B.

With a PropGraph we can now go arbitrarily deep to find comparables in other counties, states, and even countries!

This query finds all such connected properties
"""

print(conn.gsql('''
drop query comp_search

create query comp_search(vertex<property> p) for graph propgraph{
    
    start = {p};
    tgt = select t from start:s-(comparable:e)-property:t ;
    print tgt;
}

install query comp_search
'''))



Semantic Check Fails: These queries could not be found anywhere: [comp_search].
Successfully created queries: [comp_search].
Start installing queries, about 1 minute ...
comp_search query: curl -X GET 'https://127.0.0.1:9000/query/propgraph/comp_search?p=VALUE'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.
Select 'm1' as compile server, now connecting ...
Node 'm1' is prepared as compile server.

Query installation finished.


In [2]:
TARGET_PROP_ID = '72883770' # Replace with your desired property id.

q = conn.gsql('run query comp_search({})'.format(TARGET_PROP_ID))
pprint(q)


('{\n'
 '"error": false,\n'
 '"message": "",\n'
 '"version": {\n'
 '"schema": 1,\n'
 '"edition": "enterprise",\n'
 '"api": "v2"\n'
 '},\n'
 '"results": [{"tgt": [{\n'
 '"v_id": "72934629",\n'
 '"attributes": {\n'
 '"zip": "02120",\n'
 '"baths": 3,\n'
 '"address": "34 Delle Ave",\n'
 '"lng": -71.09891,\n'
 '"city": "Boston",\n'
 '"price": 1675000,\n'
 '"sqft": 2726,\n'
 '"state": "MA",\n'
 '"beds": 9,\n'
 '"type": "Multi-Family (2-4 Unit)",\n'
 '"lat": 42.33105,\n'
 '"url": "https://www.redfin.com/MA/Boston/34-Delle-Ave-02120/home/9297667"\n'
 '},\n'
 '"v_type": "property"\n'
 '}]}]\n'
 '}')


In [21]:
result = json.loads(q)
if result['error']:
    print('error', result['error'])
else:
    # Find network of comparables.
    print('comparables\n---')
    for r in result['results']:
        for tgt in r['tgt']:
            p = tgt['attributes']
            print(tgt['v_id'],':', p['address'], p['state'], p['zip'], '$' + str(p['price']))

comparables
---
72934629 : 34 Delle Ave MA 02120 $1675000
